# 분석 전 기본세팅
> 분석 전 기본세팅, 필요코드 (불러오기, 내보내기, random forest, catboost, cross validation ..)

- toc: true 
- badges: true
- comments: true
- categories: [Python]
- image: images/

---

# 파일

## 파일 불러오기

``` python
data = pd.read_csv("G:/내 드라이브/bb/cc/data.csv')
```

## 파일 내보내기

``` python
test.to_csv('test.csv', index = False)
```

- 해당 소스코드가 있는 곳에 파일이 내보내진다.

```python
test.to_csv('G:/내 드라이브/Github/TIL-Blog/test.csv', index = False)
```

- 해당 경로에 파일이 내보내진다.

---

# 기본 라이브러리

``` python
import pandas as pd # pandas

import numpy as np # numpy

import matplotlib.pyplot as plt # matplotlib
import matplotlib

import seaborn as sns # seaborn
```

---

# train test split

```python

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train[['종속변수1', '종속변수2']], train['반응변수'], test_size=0.3, shuffle=False)

# shuffle=False 추가가능. default = True

```

---

# Model

## CatBoost + 예시

``` python
from catboost import CatBoostRegressor # 캣부스트 회귀
from catboost import CatBoostClassifier # 캣부스트 분류
```

``` python
model = CatBoostRegressor()
model.fit(X_train, y_train, silent=True)

pred = model.predict(X_test)

rmse = (np.sqrt(np.mean(mean_squared_error(y_test, pred))))
rmse
```

## Random Forest

```python
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
```

```python
rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123,max_depth=6)

rf.fit(X_train, y_train)
```

## XG BOOST, LightGBM

``` python
from xgboost import XGBRegressor
from xgboost import XGBClassifier

from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
```

---

# 평가점수

## RMSE

``` python
from sklearn.metrics import mean_squared_error

rmse = (np.sqrt(np.mean(mean_squared_error(y_test, pred))))
rmse
```

## AUC

```python
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predicted)
```